Black-Litterman with Fama French Factor Modeling
Built in Lagrangian Mean Variance Optimization

In [3]:
import numpy as np
import pandas as pd
import yfinance as yf
from scipy.optimize import minimize

# Set your tickers and time window
tickers = ['AAPL', 'MSFT', 'GOOGL', 'AMZN']
start_date = "2020-01-01"
end_date = "2024-12-31"

# Download adjusted closing prices
prices = yf.download(tickers, start=start_date, end=end_date)['Adj Close']
prices.dropna(inplace=True)

# Compute daily returns
returns = prices.pct_change().dropna()

# Compute annualized covariance matrix
cov_matrix = returns.cov() * 252

[*********************100%***********************]  4 of 4 completed


KeyError: 'Adj Close'

In [4]:
# Assume equal weights for market portfolio (or use cap weights)
market_weights = np.ones(len(tickers)) / len(tickers)

# Set risk aversion parameter
risk_aversion = 2.5

# Compute implied returns π = λ Σ w_mkt
pi = risk_aversion * cov_matrix @ market_weights
pi = pd.Series(pi, index=tickers)

ValueError: Dot product shape mismatch, (0, 0) vs (4,)

In [ ]:
# Set your assumed factor risk premia
factor_premia = {
    'MKT': 0.05,
    'SMB': 0.02,
    'HML': 0.015,
    'RMW': 0.012,
    'CMA': 0.008
}
factors = list(factor_premia.keys())
factor_premia_vec = np.array(list(factor_premia.values()))

In [ ]:
# Generate random factor exposures for each ticker (replace with real estimates if available)
np.random.seed(42)
factor_betas = pd.DataFrame(
    np.random.normal(loc=0, scale=0.5, size=(len(tickers), len(factors))),
    index=tickers,
    columns=factors
)

In [ ]:
# Example: AAPL expected to outperform MSFT by FF belief spread
view_assets = [('AAPL', 'MSFT')]

# Define Q (differences in factor-based beliefs)
Q = np.array([mu_ff[a1] - mu_ff[a2] for a1, a2 in view_assets])

# Create P matrix: each row is a view, each column is an asset
P = np.zeros((len(Q), len(tickers)))
for i, (a1, a2) in enumerate(view_assets):
    P[i, tickers.index(a1)] = 1
    P[i, tickers.index(a2)] = -1

# Set confidence matrix (Ω) — diagonal, one per view
omega = np.diag([0.0004] * len(Q))

In [ ]:
# Set τ: uncertainty in prior
tau = 0.025

# Black–Litterman blending: μ* = posterior belief vector
tau_sigma_inv = np.linalg.inv(tau * cov_matrix)
omega_inv = np.linalg.inv(omega)
P_T = P.T

A = tau_sigma_inv + P_T @ omega_inv @ P
b = tau_sigma_inv @ pi.values + P_T @ omega_inv @ Q

mu_star = pd.Series(np.linalg.inv(A) @ b, index=tickers)

In [ ]:
# Compare Market Implied, Fama–French, and Black–Litterman blended returns
expected_returns = pd.DataFrame({
    'Implied Market π': pi,
    'Fama–French μ_FF': mu_ff,
    'Posterior μ* (BL)': mu_star
})

expected_returns

In [ ]:


# Objective: minimize portfolio variance
def portfolio_variance(weights, cov_matrix):
    return weights.T @ cov_matrix @ weights

# Constraint: weights must sum to 1
def constraint_sum_to_1(weights):
    return np.sum(weights) - 1

# Constraint: portfolio must hit target return
def constraint_target_return(weights, expected_returns, target_return):
    return weights @ expected_returns - target_return

# Set your target expected return (e.g., mean of mu_star)
target_return = mu_star.mean()

# Initial guess (equal weights)
n = len(tickers)
init_weights = np.ones(n) / n

# Bounds: no shorting (0 ≤ w ≤ 1)
bounds = [(0, 1) for _ in range(n)]

# Define constraints
constraints = [
    {'type': 'eq', 'fun': constraint_sum_to_1},
    {'type': 'eq', 'fun': constraint_target_return, 'args': (mu_star.values, target_return)}
]

# Run optimization
result = minimize(
    fun=portfolio_variance,
    x0=init_weights,
    args=(cov_matrix,),
    method='SLSQP',
    bounds=bounds,
    constraints=constraints
)

optimal_weights = pd.Series(result.x, index=tickers)

In [ ]:
import matplotlib.pyplot as plt

# Plot optimal portfolio weights
plt.figure(figsize=(8, 5))
optimal_weights.plot(kind='bar', color='skyblue')
plt.title('Optimal Portfolio Weights (Black–Litterman + MPT)')
plt.ylabel('Weight')
plt.xlabel('Asset')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

In [ ]:
import seaborn as sns

# Plot heatmap of covariance matrix
plt.figure(figsize=(7, 6))
sns.heatmap(cov_matrix, annot=True, fmt=".2f", cmap="YlGnBu", linewidths=0.5)
plt.title('Covariance Matrix of Annualized Returns')
plt.tight_layout()
plt.show()

In [ ]:
# Compare Implied (π), Fama-French (μ_FF), Posterior (μ*)
expected_returns.plot(kind='bar', figsize=(10, 5), rot=0)
plt.title('Return Estimates: Market vs. Fama–French vs. Posterior')
plt.ylabel('Expected Annual Return')
plt.grid(axis='y', linestyle='--', alpha=0.5)
plt.tight_layout()
plt.show()

In [ ]:
# Generate portfolios along the efficient frontier
from numpy.random import dirichlet

n_portfolios = 1000
weights_set = dirichlet(np.ones(n), size=n_portfolios)

port_returns = weights_set @ mu_star.values
port_vols = [np.sqrt(w @ cov_matrix @ w) for w in weights_set]

# Plot frontier with optimized point highlighted
plt.figure(figsize=(8, 5))
plt.scatter(port_vols, port_returns, c=port_returns/port_vols, cmap='viridis', alpha=0.6, label='Random Portfolios')
plt.scatter(np.sqrt(optimal_weights @ cov_matrix @ optimal_weights),
            optimal_weights @ mu_star.values,
            color='red', s=60, label='Optimized Portfolio')
plt.xlabel('Volatility (σ)')
plt.ylabel('Expected Return (μ)')
plt.title('Efficient Frontier with Optimized Portfolio')
plt.legend()
plt.grid(True, linestyle='--', alpha=0.4)
plt.tight_layout()
plt.show()